In [1]:
%load_ext autoreload
%autoreload 2

from coeditor.common import *
import os

from coeditor.model import CoeditorModel, _Tokenizer, DecodingArgs
from coeditor.dataset import TokenizedEditDataset
from coeditor.history import show_change
from coeditor.encoding import TokenizedEdit, decode_tokens, WindowArgs
from spot.data import output_ids_as_seqs
import shutil
import random

os.chdir(proj_root())

In [2]:
model_dir = get_model_dir(trained=False) / "small" / "checkpoint-11244"
model = CoeditorModel.load_pretrained(model_dir)
model.to("cuda:1")

In [3]:
n_samples = 50

test_data_name = "small"
test_data: TokenizedEditDataset = pickle_load(get_dataset_dir(test_data_name) / "tokenized-file_based" / "test.pkl")
samples = [e for e in test_data.all_edits() if len(e.output_tks) < 50]
random.seed(123)
random.shuffle(samples)
samples = samples[:n_samples]

In [6]:
out_dir = Path("output/inspect_coeditor_model")
shutil.rmtree(out_dir, ignore_errors=True)
out_dir.mkdir(exist_ok=True, parents=True)

decode_args = DecodingArgs()

for i, ex in enumerate(tqdm(samples)):
    wargs = WindowArgs(4096)
    ex = ex.truncate_ctx(wargs)
    pred_tks = model.predict(ex.input_tks, decode_args)

    (out_dir / f"{i}-truth.txt").write_text(ex.show())
    pred_ex = TokenizedEdit(ex.path, ex.input_tks, pred_tks)
    (out_dir / f"{i}-pred.txt").write_text(pred_ex.show())
    compare_str = ex.show_prediction(pred_tks)
    (out_dir / f"{i}-compare.txt").write_text(compare_str)


100%|██████████| 50/50 [00:55<00:00,  1.12s/it]
